In [1]:
comment  = 'Produto não recomendo não ler em nenhum dispositivo'
comment = comment.lower()


In [2]:
import re
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:

order_reviews_path = '/Users/sviat/Downloads/OneDrive_1_30-04-2024/S_Data/order_reviews.csv'

train_data = pd.read_csv(order_reviews_path)
train_data = train_data[['review_comment_message', 'review_score']]
train_data = train_data.dropna()
train_data = train_data.rename(
    columns={'review_comment_message': 'text', 'review_score': 'score'})
# Train Data Labels
train_data["score"] = train_data["score"].astype('category')
train_data["score_label"] = train_data["score"].cat.codes
train_features, train_labels = train_data['text'], tf.one_hot(
    train_data["score_label"], 5)
d = dict(enumerate(train_data["score"].cat.categories))
print(d)


vocab_size = 15000
vector_size = 100
max_seq_len = 20
tokenizer = Tokenizer(oov_token="<OOV>", num_words=vocab_size)
tokenizer.fit_on_texts(train_data['text'])
sequences_train = tokenizer.texts_to_sequences(comment)

padded_train = pad_sequences(
    sequences_train, padding='post', maxlen=max_seq_len)



model = tf.keras.models.load_model('/Users/sviat/SMART/SMART/models/lstm.keras')
p = (model.predict(padded_train)> 0.5).astype("int32")
predict = np.array(tf.argmax(p, axis=1))

2024-05-06 02:05:42.472471: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/sviat/SMART/SMART/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step


In [4]:
d[predict[0]]

1

In [5]:
data_path = '/Users/sviat/SMART/SMART/Tweets_Mg.csv'

train_data = pd.read_csv(data_path)
train_data = train_data[['Text', 'Classificacao']]
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.rename(
    columns={'Text': 'text', 'Classificacao': 'score'})


for i in range(len(train_data)):
    train_data.iloc[i]['text'] = train_data.iloc[i]['text'].lower()

for i in range(len(train_data)):
    train_data.iloc[i]['text'] = re.sub(
        pattern=r'[\w]*[^\w\s][\w]*', repl=' ', string=train_data.iloc[i]['text'])
# Train Data Labels
train_data["score"] = train_data["score"].astype('category')
train_data["score_label"] = train_data["score"].cat.codes
train_features, train_labels = train_data['text'], tf.one_hot(
    train_data["score_label"], 3)
d = dict(enumerate(train_data["score"].cat.categories))
print(d)


vocab_size = 15000
vector_size = 300
max_seq_len = 32
tokenizer = Tokenizer(oov_token="<OOV>", num_words=vocab_size)
tokenizer.fit_on_texts(train_data['text'])
sequences_train = tokenizer.texts_to_sequences(train_data['text'])


model = tf.keras.models.load_model('/Users/sviat/SMART/SMART/models/tweets_lstm1.keras')
model.summary()
p = (model.predict(padded_train)> 0.5).astype("int32")
predict = np.array(tf.argmax(p, axis=1))


/var/folders/mb/w9_4bn093gbdkrx6gk7v40040000gn/T/ipykernel_27365/2876896554.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.iloc[i]['text'] = train_data.iloc[i]['text'].lower()
/var/folders/mb/w9_4bn093gbdkrx6gk7v40040000gn/T/ipykernel_27365/2876896554.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.iloc[i]['text'] = re.sub(


{0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 32, 300)          │     4,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 32, 300)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 32, 32)           │        42,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (32, 3)                │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 3)                │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,629,206 (51.99 MB)

 Trainable params: 4,543,068 (17.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,086,138 (34.66 MB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 461ms/step


In [6]:
d[predict[0]]

'Neutro'